In [25]:
import yfinance as yf 
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.nn import Module,Sequential,Conv1d,ReLU,Dropout,MaxPool1d,Linear, BatchNorm1d
import torch.nn.functional as F
from torch.nn import MSELoss , L1Loss
from torch.optim import Adam
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
#from torch import mean
from math import sqrt
import numpy as np
from statistics import mean
from sklearn.model_selection import TimeSeriesSplit
import copy

In [26]:
LOOKBACK = 10
HORIZON = 5

In [27]:
gps_price = yf.download('gps', start = '2001-01-31', end = '2021-10-31', time_interval = 'daily')
gps_price = gps_price['Close']
scaler = MinMaxScaler(feature_range=(0, 1))
gps_scaler = scaler.fit_transform(gps_price.values.reshape(-1,1))

[*********************100%***********************]  1 of 1 completed


In [28]:
gps_scaler

array([[0.66317839],
       [0.65952783],
       [0.66926259],
       ...,
       [0.39571672],
       [0.42492088],
       [0.41834997]])

In [29]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.9 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [30]:
def split_data(lookback,horizon, price):
    data_initial = price
    window_size = lookback + horizon
    data = []
    for index in range(len(data_initial) - window_size):
        data.append(data_initial[index:index+ window_size])
    data = np.array(data) 

    # test_size = int(np.round(0.2*data.shape[0]))
    # train_set_size = data.shape[0] - (test_size)
    train_set = data[:]
    
    x_train = train_set[:,:lookback]
    y_train = train_set[:,lookback:lookback + horizon]
    y_train = y_train.reshape(-1,horizon)

    # test_set = data[train_set_size:]
    # x_test = test_set[:,:lookback]
    # y_test = test_set[:,lookback:lookback+ horizon]
    # y_test = y_test.reshape(-1,horizon)
    
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[2],x_train.shape[1])
    #y_train = y_train.reshape(y_train.shape[0],y_train.shape[2],y_train.shape[1])
    # x_test = x_test.reshape(x_test.shape[0],x_test.shape[2],x_test.shape[1])
    #y_test = y_test.reshape(y_test.shape[0],y_test.shape[2],y_test.shape[1])


    return [x_train, y_train]

In [31]:
class cnn_dataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y 
    def __len__(self):
        return len(self.X)
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

In [32]:
device = 'cpu'

In [33]:
class PrintLayer(Module):
    def forward(self,X):
        print(X.shape)
        return X
class cnn_1(Module):
    CNN:Sequential
    FC:Sequential

    def __init__(self) -> None:
        Module.__init__(self)
        
        cnn_layers = []
      
        cnn_layers.append(Conv1d(in_channels = 1,out_channels =64,kernel_size =3,stride =1 ,padding = 1))         
        cnn_layers.append(ReLU())
        #cnn_layers.append(MaxPool1d(2))
        cnn_layers.append(Dropout(0.5))
        

       

        cnn_layers.append(Conv1d(in_channels =64,out_channels = 32, kernel_size = 3, padding = 1))
        cnn_layers.append(ReLU())
        # cnn_layers.append(Dropout(0.5))
        # cnn_layers.append(MaxPool1d(2))
        

        
        #cnn_layers.append(Conv1d(in_channels =32,out_channels = 8, kernel_size = 3))
        #cnn_layers.append(ReLU())

        fc_layers  = []
        #fc_layers.append(PrintLayer())

        fc_layers.append(Linear(in_features =320,out_features = 128))  
        fc_layers.append(ReLU())
        fc_layers.append(Dropout(0.5))
        #fc_layers.append(BatchNorm1d(32))

        fc_layers.append(Linear(in_features  = 128, out_features = HORIZON))  
        fc_layers.append(ReLU())
       

        self.CNN = Sequential(*cnn_layers)
        
        self.FC = Sequential(*fc_layers)

    def forward(self,X)->None:
        
        output = self.CNN(X)
        output = torch.flatten(output,start_dim=1)
        output = self.FC(output)

        return output

In [34]:
train_losses = []
valid_losses = []

In [35]:
def Train(data_loader, model,optimizer, criterion,) :
    running_loss = 0
    
    for (X,y) in (data_loader):
        X = X.to(device)
        y = y.to(device)
       
        #l2_lambda = 0.001
        #l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        #loss = loss + l2_lambda *l2_norm
        optimizer.zero_grad()
        preds = model(X.float())
        loss = criterion(preds, y.float())
       
        
        loss.backward()
        optimizer.step()
        running_loss += np.sqrt(loss.item())

    train_loss = running_loss/len(data_loader)
    # print(f'train_loss{train_loss}')

    return train_loss, model

    



def Valid(data_loader,  model, optimizer, criterion):
    """
    return y_true, y_hat
    """
    running_loss = 0
    
    
    y_hat = []
    y_true = []
    with torch.no_grad():
        for (X, y) in (data_loader):
            X = X.to(device)
            y = y.to(device)
         
            y_true = [*y_true,*(y.reshape(-1).tolist()) ]
            optimizer.zero_grad()
            preds = model(X.float())
         
            y_hat = [*y_hat, *(preds.reshape(-1).tolist())]
      
            loss = criterion(preds,y.float())
            running_loss += np.sqrt(loss.item())
            
        valid_loss = running_loss/len(data_loader)
        
        # print(f'valid_loss {valid_loss}')

    return y_true, y_hat, valid_loss, model


In [36]:
spliter = BlockingTimeSeriesSplit(n_splits = 10)

cv_test_losses = []
cv_train_losses = []
cv_val_losses = []

for trainval_idx ,test_idx in spliter.split(gps_scaler):
    # print(trainval_idx)
    # print(test_idx)

    train_idx = trainval_idx[ : int(len(trainval_idx)*0.8)]
    val_idx   = trainval_idx[int(len(trainval_idx)*0.8) : ]


    x_train_unsplit = gps_scaler[train_idx]
    x_val_unsplit = gps_scaler[val_idx]
    x_test_unsplit = gps_scaler[test_idx]

    # print(train_idx)
    # print(val_idx)

    x_train_gps, y_train_gps = split_data(LOOKBACK, HORIZON, x_train_unsplit)
    x_val_gps, y_val_gps = split_data(LOOKBACK, HORIZON, x_val_unsplit)
    x_test_gps, y_test_gps = split_data(LOOKBACK, HORIZON, x_test_unsplit)
    # print(x_train_gps.shape)
    # print(y_train_gps.shape)

    # print(x_val_gps.shape)
    # print(y_val_gps.shape)

    # print(x_test_gps.shape)
    # print(y_test_gps.shape)


    train_dataset_gps =cnn_dataset(x_train_gps,y_train_gps)
    train_loader_gps = torch.utils.data.DataLoader(train_dataset_gps, batch_size = 16, shuffle = False)
    test_dataset_gps = cnn_dataset(x_test_gps,y_test_gps)
    test_loader_gps = torch.utils.data.DataLoader(test_dataset_gps,batch_size=16,shuffle=False)
    valid_dataset_gps = cnn_dataset(x_val_gps,y_val_gps)
    valid_loader_gps = torch.utils.data.DataLoader(valid_dataset_gps,batch_size=16,shuffle=False)
    
    model = cnn_1()
    model = model.to(device)
    criterion = MSELoss(reduction= 'mean')
    optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4, weight_decay= 1e-10)

    train_losses = []
    val_losses = []

    epochs = 200

    best_val_loss = 1e+9

    for epoch in range(epochs):
        # print('epochs {}/{}'.format(epoch+1,epochs))
        train_loss , model               = Train(train_loader_gps, model, optimizer, criterion)
        y_true, y_hat, valid_loss , model = Valid(valid_loader_gps, model, optimizer, criterion)
    
        train_losses.append(train_loss)
        val_losses.append(valid_loss)

        curr_val_loss = valid_loss

        if curr_val_loss < best_val_loss:
            best_model = copy.deepcopy(model)
            best_val_loss = curr_val_loss
            print("Best model : ", epoch)

    
    y_test_true, y_test_hat, test_loss , _ = Valid(test_loader_gps, best_model, optimizer, criterion)

    print(test_loss)
    cv_test_losses.append(test_loss)
    cv_train_losses.append(train_losses)
    cv_val_losses.append(val_losses)

print(cv_test_losses)
print(sum(cv_test_losses) / len(cv_test_losses))

Best model :  0
Best model :  1
Best model :  2
Best model :  3
Best model :  4
Best model :  5
Best model :  6
Best model :  9
Best model :  11
Best model :  12
Best model :  15
Best model :  17
Best model :  18
Best model :  24
Best model :  25
Best model :  26
Best model :  27
Best model :  28
Best model :  36
Best model :  48
0.0494861269689246
Best model :  0
Best model :  1
Best model :  2
Best model :  3
Best model :  4
Best model :  5
Best model :  7
Best model :  8
Best model :  9
Best model :  11
Best model :  12
Best model :  14
Best model :  17
Best model :  20
Best model :  22
Best model :  26
Best model :  28
Best model :  29
Best model :  34
Best model :  36
Best model :  39
Best model :  46
Best model :  49
Best model :  57
Best model :  58
Best model :  59
Best model :  67
Best model :  75
Best model :  91
Best model :  92
Best model :  93
Best model :  105
Best model :  115
Best model :  123
Best model :  131
Best model :  147
Best model :  149
Best model :  153
Best 